In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import malaya_speech
import random
import numpy as np
import malaya
from sklearn.utils import shuffle
from sklearn.utils.random import sample_without_replacement

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
malaya_speech.force_alignment.transducer.available_transformer()

,Size (MB),Quantized Size (MB),Language
conformer-transducer,120,32.3,[malay]
conformer-transducer-mixed,120,32.3,"[malay, singlish]"
conformer-transducer-singlish,120,32.3,[singlish]


In [4]:
model = malaya_speech.force_alignment.transducer.transformer(model = 'conformer-transducer', device = 'gpu:0')
singlish_model = malaya_speech.force_alignment.transducer.transformer(model = 'conformer-transducer-singlish', device = 'gpu:0')

2023-04-09 23:57:15.768625: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 23:57:15.837446: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-09 23:57:15.852094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-09 23:57:15.853080: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [5]:
asr = malaya_speech.stt.transducer.pt_transformer(model = 'mesolitica/conformer-medium-mixed')
_ = asr.cuda()
_ = asr.eval()

In [6]:
lm = malaya.language_model.kenlm(model = 'bahasa-wiki-news-iium-stt')

In [7]:
lm.score('tak suka hg')

-13.193544387817383

In [8]:
import json
from glob import glob
from tqdm import tqdm

with open('/home/husein/ssd1/speech-bahasa/malay-asr-train.json') as fopen:
    ms = json.load(fopen)

In [9]:
with open('/home/husein/dev/malaya-speech/semisupervised-transducer.json') as fopen:
    semi = json.load(fopen)
    
filtered_X, filtered_Y = [], []
for i in range(len(semi['X'])):
    if semi['Y'][i][1] > 0.2 and len(semi['Y'][i][0]):
        filtered_X.append(semi['X'][i])
        filtered_Y.append(semi['Y'][i][0])
        
ms['X'] = ms['X'] + filtered_X
ms['Y'] = ms['Y'] + filtered_Y

In [10]:
with open('/home/husein/dev/malaya-speech/semisupervised-transducer-v2.json') as fopen:
    semi = json.load(fopen)
    
filtered_X, filtered_Y = [], []
for i in range(len(semi['X'])):
    if semi['Y'][i][1] > 0.2 and len(semi['Y'][i][0]):
        filtered_X.append(semi['X'][i])
        filtered_Y.append(semi['Y'][i][0])
        
ms['X'] = ms['X'] + filtered_X
ms['Y'] = ms['Y'] + filtered_Y

In [11]:
with open('/home/husein/malaya-speech/singlish-stt-train.json') as fopen:
    sg = json.load(fopen)

In [12]:
len(sg['X'])

3284901

In [13]:
sg['X'][:10]

['/home/husein/ssd2/imda/wav/5-75-tfrecord-1409.wav',
 '/home/husein/ssd2/imda/wav/5-118-tfrecord-1786.wav',
 '/home/husein/ssd2/imda/wav/4-77-tfrecord-2083.wav',
 '/home/husein/ssd2/imda/wav/2-39-tfrecord-78.wav',
 '/home/husein/ssd2/imda/wav/7-63-tfrecord-730.wav',
 '/home/husein/ssd2/imda/wav/5-68-tfrecord-1531.wav',
 '/home/husein/ssd2/imda/wav/0-17-tfrecord-3636.wav',
 '/home/husein/ssd2/imda/wav/1-113-tfrecord-1496.wav',
 '/home/husein/ssd2/imda/wav/7-108-tfrecord-1945.wav',
 '/home/husein/ssd2/imda/wav/5-63-tfrecord-3858.wav']

In [14]:
from datasets import Audio

sr = 16000
minimum = int(0.3 * sr)
audio = Audio(sampling_rate=sr)

In [16]:
!ls

augmentation-switchboard-v18	    augment-switchboard-lm-v2.ipynb
augmentation-switchboard-v20	    augment-switchboard-v2.ipynb
augment-switchboard-lm-Copy1.ipynb  augment-switchboard-v3.ipynb
augment-switchboard-lm-Copy2.ipynb  augment-switchboard-v4.ipynb
augment-switchboard-lm.ipynb	    augment-switchboard-v5.ipynb
augment-switchboard-lm-tts.ipynb    README.md


In [44]:
directory = 'augmentation-switchboard-v21'
!mkdir {directory}

In [45]:
import torchaudio
import torch
from itertools import permutations

In [46]:
def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [47]:
def groupby(alignment, length, min_threshold = 0.5):
    r = []
    g = []
    for no, row in enumerate(alignment):
        
        if no > 0 and len(r) and alignment[no]['start'] - alignment[no-1]['end'] >= min_threshold:
            g.append(r)
            r = []
        
        r.append(row)

    return g

In [ ]:
data = {}
for O in tqdm(range(20000)):
    try:
        i_ms = sample_without_replacement(len(ms['X']), 4)
        i_sg = sample_without_replacement(len(sg['X']), 4)

        groups = []
        for i in i_ms:
            x = ms['X'][i]
            v = ms['Y'][i]
            y = audio.decode_example(audio.encode_example(x))['array']
            asr_ = asr.forward([y])[0]
            
            if asr_[0][1][0] < 0.15 and calculate_wer(v, asr_[0][0]) > 0.1:
                # print(f'skip ms {i}', v, asr_[0][0], asr_[0][1][0])
                continue
                
            result = model.predict(y, v)

            if ' '.join([r['text'] for r in result['words_alignment']]) != v:
                continue

            grouped = groupby(result['words_alignment'], len(y) / sr)
            # print(grouped)
            for g in grouped:
                y_ = y[int(g[0]['start'] * sr): int(g[-1]['end'] * sr) + minimum]
                if len(y_):
                    v_ = [g_['text'] for g_ in g]
                    groups.append((y_, v_))

        for i in i_sg:
            x = sg['X'][i]
            v = sg['Y'][i]
            y = audio.decode_example(audio.encode_example(x))['array']
            asr_ = asr.forward([y])[0]
            
            if asr_[0][1][0] < 0.15 and calculate_wer(v, asr_[0][0]) > 0.1:
                # print(f'skip sg {i}', v, asr_[0][0], asr_[0][1][0])
                continue
                
            result = singlish_model.predict(y, v)

            if ' '.join([r['text'] for r in result['words_alignment']]) != v:
                continue

            grouped = groupby(result['words_alignment'], len(y) / sr)
            for g in grouped:
                y_ = y[int(g[0]['start'] * sr): int(g[-1]['end'] * sr) + minimum]
                if len(y_):
                    v_ = [g_['text'] for g_ in g]
                    groups.append((y_, v_))
                    
        if not len(groups):
            continue
            
        groups = sorted(groups, key = lambda x: len(' '.join(x[1])), reverse = True)[:7]
        
        strings = [' '.join(g[1]) for g in groups]
        perm = list(permutations(range(len(groups))))
        
        scores = []
        for p in perm:
            s = ' '.join([strings[p_] for p_ in p])
            scores.append(lm.score(s))
            
        best_perm = perm[np.argmax(scores)]
        
        l = 0
        combine_y, combine_v = [], []
        index = 0
        while l < 15 and index < len(groups):
            g = groups[best_perm[index]]
            l_ = len(g[0]) / sr
            if l_ < 1.0:
                index += 1
                continue
            r_int = random.randint(3000, 16000)
            l += l_ + (r_int / sr)
            combine_y.append(g[0] / np.abs(g[0]).max())
            combine_y.append(np.zeros(shape = (r_int,)))
            combine_v.extend(g[1])
            index += 1

        if len(combine_v):
            audio_path = f'{directory}/{O}.mp3'
            torchaudio.save(audio_path, 
                            torch.tensor(np.concatenate(combine_y).astype('float32')).unsqueeze(0), 
                            sr, format='mp3')
            data[O] = ' '.join(combine_v)
            
    except Exception as e:
        pass

 17%|█████▌                           | 3374/20000 [2:18:02<11:11:16,  2.42s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 95%|████████████████████████████████▏ | 18925/20000 [13:01:33<43:51,  2.45s/it]

In [49]:
len(data)

14373

In [50]:
data

{0: 'jadi saya bantu isteri saya',
 1: 'its just gods plan dont need to think so much yang tetap berpegang teguh dengan dua jalan ini',
 3: 'juga mempunyai pilihan pilihan yang yang lebih banyaklah yang berlaku bahawa',
 4: 'dan perancangan strategik pdps to saint johns sebelum membawa pindaan ini berkaitan dengan pendaftaran ini oleh kerana india banyak menulis buku tentang sanskrit di sebelah sini di sebelah',
 5: 'sehinggalah dapat melepasi bahaya barulah dia periksa jumlah keseluruhan mereka tak sempat nak kira semua ahli keluarga yang besar dan tokoh tokoh media yang mengagumi after a hot day',
 6: 'imagine the government taking away that and making it illegal completely so you can imagine how much black market',
 7: 'ummi memandang sahaja kepada siti salina tidak juga bersuara',
 8: 'wahai rasulullah sallallahu alaihi salam andai sekiranya aku tidak dapati kamu di situ',
 10: 'walaupun dalam teori mereka boleh melakukan setiap tahun selagi mereka berada di dalam negara negara yan

In [51]:
import IPython.display as ipd
ipd.Audio(f'{directory}/13.mp3')

In [52]:
directory

'augmentation-switchboard-v21'

In [53]:
with open('augment-switchboard-v21.json', 'w') as fopen:
    json.dump(data, fopen)